In [1]:
import arviz as az
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import seaborn.objects as so
import numpy as np
import pandas as pd
import polars as pl

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

RANDOM_SEED = 504




In [4]:
from retrieve_sql_module import get_pbp_data

years = list(range(2021,2025))
pbp_data = get_pbp_data(years, 0)

In [5]:
pbp_data

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle
0,FF,2021-10-03,92.3,1.40,6.80,"Smith, Will",596019,519293,field_out,hit_into_play,...,1,3,3.0,1.0,6.0,NaN,1.28,0.69,-0.69,47.4
1,SL,2021-10-03,80.6,1.60,6.64,"Smith, Will",596019,519293,None,foul,...,1,3,3.0,1.0,6.0,NaN,2.99,-0.77,0.77,44.3
2,CU,2021-10-03,75.5,1.46,6.88,"Smith, Will",596019,519293,None,foul,...,1,3,3.0,1.0,6.0,NaN,4.52,-0.65,0.65,51.7
3,CU,2021-10-03,75.0,1.53,6.83,"Smith, Will",596019,519293,None,ball,...,1,3,3.0,1.0,6.0,NaN,4.74,-0.69,0.69,49.5
4,FF,2021-10-03,91.2,1.49,6.66,"Smith, Will",607043,519293,field_out,hit_into_play,...,1,3,3.0,1.0,6.0,NaN,1.49,0.63,0.63,44.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2891004,FF,2024-03-20,95.7,-2.13,5.77,"Darvish, Yu",660271,506433,None,ball,...,1,0,NaN,NaN,8.0,1.0,1.03,0.54,-0.54,NaN
2891005,SI,2024-03-20,93.9,-2.17,5.70,"Darvish, Yu",605141,506433,None,ball,...,1,0,NaN,NaN,8.0,1.0,1.57,1.34,1.34,NaN
2891006,SI,2024-03-20,93.4,-2.00,5.70,"Darvish, Yu",605141,506433,None,ball,...,1,0,NaN,NaN,8.0,1.0,1.63,1.42,1.42,NaN
2891007,FF,2024-03-20,92.6,-1.94,5.83,"Darvish, Yu",605141,506433,None,foul,...,1,0,NaN,NaN,8.0,1.0,1.11,0.25,0.25,NaN


In [5]:
singles_result = (
    pbp_data[pbp_data['events'] == 'single']
    .groupby(['batter','game_year'])
    .size()
    .reset_index(name='single_n')
)

In [6]:
singles_result

,batter,game_year,single_n
0,405395,2021,45
1,405395,2022,45
2,408234,2021,90
3,408234,2022,86
4,408234,2023,62
...,...,...,...
2818,808982,2024,35
2819,810938,2024,1
2820,813860,2024,1
2821,814816,2024,1


In [11]:
pbp_data['single_n'] = (
    pbp_data.groupby(['batter', 'game_year'])['events']
    .transform(lambda x: (x == 'single').sum())
)

In [12]:
pbp_data

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle,single_n
0,FF,2021-10-03,92.3,1.40,6.80,"Smith, Will",596019,519293,field_out,hit_into_play,...,3,3.0,1.0,6.0,NaN,1.28,0.69,-0.69,47.4,65
1,SL,2021-10-03,80.6,1.60,6.64,"Smith, Will",596019,519293,None,foul,...,3,3.0,1.0,6.0,NaN,2.99,-0.77,0.77,44.3,65
2,CU,2021-10-03,75.5,1.46,6.88,"Smith, Will",596019,519293,None,foul,...,3,3.0,1.0,6.0,NaN,4.52,-0.65,0.65,51.7,65
3,CU,2021-10-03,75.0,1.53,6.83,"Smith, Will",596019,519293,None,ball,...,3,3.0,1.0,6.0,NaN,4.74,-0.69,0.69,49.5,65
4,FF,2021-10-03,91.2,1.49,6.66,"Smith, Will",607043,519293,field_out,hit_into_play,...,3,3.0,1.0,6.0,NaN,1.49,0.63,0.63,44.0,67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2891004,FF,2024-03-20,95.7,-2.13,5.77,"Darvish, Yu",660271,506433,None,ball,...,0,NaN,NaN,8.0,1.0,1.03,0.54,-0.54,NaN,108
2891005,SI,2024-03-20,93.9,-2.17,5.70,"Darvish, Yu",605141,506433,None,ball,...,0,NaN,NaN,8.0,1.0,1.57,1.34,1.34,NaN,91
2891006,SI,2024-03-20,93.4,-2.00,5.70,"Darvish, Yu",605141,506433,None,ball,...,0,NaN,NaN,8.0,1.0,1.63,1.42,1.42,NaN,91
2891007,FF,2024-03-20,92.6,-1.94,5.83,"Darvish, Yu",605141,506433,None,foul,...,0,NaN,NaN,8.0,1.0,1.11,0.25,0.25,NaN,91


In [10]:
def update_columns(data_frame):
    unique_events = ['field_out', 'strikeout', 'walk', 'single', 'double', 'triple', 'home_run', 'force_out', 'grounded_into_double_play',
                    'hit_by_pitch', 'field_error','sac_fly']
    hit_events = ['single','double','triple','home_run']
    out_events = ['double_play', 'field_error','field_out','fielders_choice_out','force_out','grounded_into_double_play','other_out','strikeout',
                  'strikeout_double_play','triple_play']
    on_base_events = ['walk', 'hit_by_pitch','sac_fly','sac_bunt']

    data_frame['hit_n'] = 0
    data_frame['out_n'] = 0
    data_frame['ob_n'] = 0
    
    for event in unique_events:
        data_frame[f'{event}_n'] = data_frame.groupby(['batter', 'game_year'])['events'].transform(lambda x: (x == event).sum())
        
    for event in hit_events:
        data_frame['hit_n'] += data_frame.groupby(['batter','game_year'])['events'].transform(lambda x: (x == event).sum())
    for event in out_events:
        data_frame['out_n'] += data_frame.groupby(['batter','game_year'])['events'].transform(lambda x: (x == event).sum())
    for event in on_base_events:
        data_frame['ob_n'] += data_frame.groupby(['batter','game_year'])['events'].transform(lambda x: (x == event).sum())
        
    data_frame['n_pitches'] = data_frame.groupby(['batter','game_year'])['events'].transform('size')
    data_frame['total_events'] = data_frame['hit_n'] + data_frame['out_n'] + data_frame['ob_n']

    data_frame['single_rate'] = data_frame['single_n'] / data_frame['total_events']
    data_frame['double_rate'] = data_frame['double_n'] / data_frame['total_events']
    data_frame['triple_rate'] = data_frame['triple_n'] / data_frame['total_events']
    data_frame['home_run_rate'] = data_frame['single_n'] / data_frame['total_events']
    data_frame['strikeout_rate'] = data_frame['strikeout_n'] / data_frame['total_events']
    data_frame['field_out_rate'] = (data_frame['out_n'] - data_frame['strikeout_n']) / data_frame['total_events']

    return data_frame

In [11]:
pbp_data = update_columns(pbp_data)

In [ ]:
from retrieve_sql_module import upload_to_sql

upload_to_sql(pbp_data, "statcast_all")